# Intelligent Defaults and Logging Configuration in SageMakerCore

---
## Introductions

In this notebook, we will walkthrough the setup and usage of intelligent defaults in the SageMakerCore SDK. Additionally, this notebook contains a section with the steps required for configuring logging levels to assist in the debugging of issues that arise while using the SDK.



### Intelligent Defaults

Intelligent Defaults is a feature provided by the SageMakerCore SDK to assist users in defining default values to be auto populated into the AWS API Request parameters. For example, if a user/admin wants all of their AWS Resources to use a specific VPC Config during creation this can be defined in the Intelligent Defaults Configs. Intelligent Defaults supports:
1. GlobalDefaults - default values applied across SageMaker API calls
2. Resource Specific Defaults - defaults applied only when creating a specific resource

An Example of the strucuture of the Intelligent Defaults Config is below:

```json
{
    "SchemaVesion": "1.0",
    "SageMaker": {
        "PythonSDK": {
            "Resources": {
                "GlobalDefaults": {
                    "vpc_config": {
                        "security_group_ids": [
                            "sg-xxxxxxxxxxxxxxxxx" // Replace with security group id
                        ],
                        "subnets": [
                            "subnet-xxxxxxxxxxxxxxxxx", // Replace with subnet id
                            "subnet-xxxxxxxxxxxxxxxxx" // Replace with subnet id
                        ]
                    }
                    // ...
                },
                "TrainingJob": {
                    "role_arn": "arn:aws:xxxxxxxxxxx:role/xxxxx", // Replace with role arn
                    "output_data_config": {
                        "s3_output_path": "s3://xxxxxxxxxxx", // Replace with S3 URI
                    },
                    // ...
                }
            }
        }
    }
}
```

### Logging Levels

To assist in debugging issues originating within the SDK, SageMakerCore provides a simple utility method - `configure_logging()`

To set the logging level users have 2 options:
1. Pass a string parameter to utility method with log level they desire - `configure_logging("DEBUG")`
2. Set the `LOG_LEVEL=INFO` environment variable and call `configure_logging()` without a parameter


In a later section in this notebook, we will walk through an example of how these options would look like in practice for a user.

## Pre-Requisites

### Install Latest SageMakerCore
All SageMakerCore beta distributions will be released to a private s3 bucket. After being allowlisted, run the cells below to install the latest version of SageMakerCore from `s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz`

Ensure you are using a kernel with python version >=3.8

In [1]:
# Uninstall previous version of sagemaker_core and restart kernel
!pip uninstall sagemaker_core -y

Found existing installation: sagemaker-core 0.1.1
Not uninstalling sagemaker-core at /Users/nargokul/workspace/sagemaker-core/src, outside environment /Users/nargokul/workspace/sagemaker-core/venv
Can't uninstall 'sagemaker-core'. No files were found to uninstall.


In [2]:
# Make dist/ directory to hold the sagemaker_core beta distribution file
!mkdir dist

mkdir: dist: File exists


In [3]:
# Download and Install the latest version of sagemaker_core
!aws s3 cp s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz dist/

!pip install dist/sagemaker_core-latest.tar.gz

download: s3://sagemaker-core-beta-artifacts/sagemaker_core-latest.tar.gz to dist/sagemaker_core-latest.tar.gz
Processing ./dist/sagemaker_core-latest.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sagemaker-core: filename=sagemaker_core-0.1.6-py3-none-any.whl size=357502 sha256=80997a679fc4e8fb199a3d99b4fc2c9407fbcc1c830ce2ab0825c77ecfe5c20c
  Stored in directory: /Users/nargokul/Library/Caches/pip/wheels/fe/f8/67/5100a16dc470a3f7e8cce4f09cd4d98128e5d4f258b4fe90aa
Successfully built sagemaker-core
  Attempting uninstall: sagemaker-core
    Found existing installation: sagemaker-core 0.1.1
    Not uninstalling sagemaker-core at /Users/nargokul/workspace/sagemaker-core/src, outside environment /Users/nargokul/workspace/sagemaker-core/venv
    Can't uninstall 'sagemaker-core'. No files were found to uninstall.

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice]

In [4]:
# Check the version of sagemaker_core
!pip show -v sagemaker_core

Name: sagemaker-core
Version: 0.1.1
Summary: An python package for sagemaker core functionalities
Home-page: 
Author: 
Author-email: AWS <sagemaker-interests@amazon.com>
License: 
Location: /Users/nargokul/workspace/sagemaker-core/src
Requires: boto3, jsonschema, platformdirs, pydantic, PyYAML
Required-by: 
Metadata-Version: 2.1
Installer: 
Classifiers:
  Development Status :: 3 - Alpha
  Intended Audience :: Developers
  License :: OSI Approved :: Apache Software License 2.0
  Programming Language :: Python :: 3.8
  Programming Language :: Python :: 3.9
  Programming Language :: Python :: 3.10
Entry-points:
Project-URLs:
  Repository, https://github.com/aws/sagemaker-core.git


### Install Additional Packages

In [5]:
# Install additionall packages

!pip install -U scikit-learn pandas boto3


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### Setup

Let's start by specifying:
- AWS region.
- The IAM role arn used to give learning and hosting access to your data. Ensure your enviornment has AWS Credentials configured.
- The S3 bucket that you want to use for storing training and model data.

In [6]:
from sagemaker_core.helper.session_helper import Session, get_execution_role
from rich import print

# Get region, role, bucket

sagemaker_session = Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
print(role)

arn:aws:iam::211125564141:role/Admin

### Load and Prepare Dataset
For this example, we will be using the IRIS data set from `sklearn.datasets` to train our XGBoost container.

In [7]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import pandas as pd

# Get IRIS Data

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [8]:
import os

# Prepare Data

os.makedirs('./data', exist_ok=True)

iris_df = iris_df[['target'] + [col for col in iris_df.columns if col != 'target']]

train_data, test_data = train_test_split(iris_df, test_size=0.2, random_state=42)

train_data.to_csv('./data/train.csv', index=False, header=False)

### Upload Data to S3
In this step, we will upload the train and test data to the S3 bucket configured earlier using `sagemaker_session.default_bucket()`

In [9]:
# Upload Data

prefix = "DEMO-scikit-iris"
TRAIN_DATA = "train.csv"
DATA_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    DATA_DIRECTORY, bucket=bucket, key_prefix="{}/{}".format(prefix, DATA_DIRECTORY)
)

s3_input_path = "s3://{}/{}/data/{}".format(bucket, prefix, TRAIN_DATA)
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

print(s3_input_path)
print(s3_output_path)

s3://sagemaker-us-west-2-211125564141/DEMO-scikit-iris/data/train.csv

s3://sagemaker-us-west-2-211125564141/DEMO-scikit-iris/output

### Fetch the XGBoost Image URI
In this step, we will fetch the XGBoost Image URI we will use as an input parameter when creating an AWS TrainingJob

In [10]:
# Image name is hardcoded here
# Image name can be programatically got by using sagemaker package and calling image_uris.retrieve
# Since that is a high level abstraction that has multiple dependencies, the image URIs functionalities will live in sagemaker (V2)

image = "433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest"

## Intelligent Defaults

### Create Intelligent Defaults JSON

In order for SageMakerCore to pick up the Intelligent Defaults Configs to populate API calls, we first must create the json config file and set the `SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE` enviornment variable.

Below we will create the config file at `data/defaults.json` and assign this path to the `SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE` enviornment variable.

In [11]:
import os
import json

DEFAULTS_CONTENT = {
    "SchemaVesion": "1.0",
    "SageMaker": {
        "PythonSDK": {
            "Resources": {
                "GlobalDefaults": {
                    "vpc_config": {
                        "security_group_ids": [
                            "sg-xxxxxxxxxxxxxxxxx" # Replace with security group id
                        ],
                        "subnets": [
                            "subnet-xxxxxxxxxxxxxxxxx", # Replace with subnet id
                            "subnet-xxxxxxxxxxxxxxxxx" # Replace with subnet id
                        ]
                    }
                },
                "TrainingJob": {
                    "role_arn": role,
                    "output_data_config": {
                        "s3_output_path": s3_output_path
                    },
            
                }
            }
        }
    }
}

path_to_defaults = os.path.join(DATA_DIRECTORY, "defaults.json")
with open(os.path.join(DATA_DIRECTORY, "defaults.json"), "w") as f:
    json.dump(DEFAULTS_CONTENT, f, indent=4)  

In [12]:
import os
# Setting path of Config file in environment variable 
os.environ['SAGEMAKER_CORE_ADMIN_CONFIG_OVERRIDE'] = path_to_defaults

### Using GlobalDefaults
In the below example,  a `Cluster` resource will be created using the `vpc_config` defined under the `SageMaker.PythonSDK.Resources.GlobalDefaults`.

In [13]:
import time
from sagemaker_core.generated.resources import Cluster
from sagemaker_core.generated.shapes import ClusterInstanceGroupSpecification, ClusterLifeCycleConfig
    
cluster_name_v3 = 'xgboost-cluster-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Use vpc_config from Intelligent Defaults JSON config file under the SageMaker.PythonSDK.Resources.GlobalDefaults key
cluster = Cluster.create(
    cluster_name=cluster_name_v3,
    instance_groups=[
        ClusterInstanceGroupSpecification(
            instance_count=1, 
            instance_group_name="instance-group-11",
            instance_type="ml.m5.4xlarge",
            life_cycle_config=ClusterLifeCycleConfig(source_s3_uri=s3_input_path, on_create="dothis"),
            execution_role=role
        )
    ]
)

/Users/nargokul/workspace/sagemaker-core/venv/lib/python3.9/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[08/07/24 19:08:10] INFO     INFO:sagemaker_core.generated.resources:Creating cluster resource.   ]8;id=92420;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/resources.py\resources.py]8;;\:]8;id=142115;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/resources.py#3097\3097]8;;\

                    WARNING  WARNING:sagemaker_core.generated.utils:No boto3 session provided.         ]8;id=367473;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py\utils.py]8;;\:]8;id=788621;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py#194\194]8;;\
                             Creating a new session.                                                               

                    WARNING  WARNING:sagemaker_core.generated.utils:No region provided. Using default  ]8;id=964649;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py\utils.py]8;;\:]8;id=868941;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py#198\198]8;;\
                             region.                                                                               

                    WARNING  WARNING:sagemaker_core.generated.utils:No config provided. Using default  ]8;id=846838;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py\utils.py]8;;\:]8;id=514741;file:///Users/nargokul/workspace/sagemaker-core/src/sagemaker_core/generated/utils.py#202\202]8;;\
                             config.                                                                               

                    INFO     INFO:botocore.credentials:Found credentials in shared credentials  ]8;id=376488;file:///Users/nargokul/workspace/sagemaker-core/venv/lib/python3.9/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=972536;file:///Users/nargokul/workspace/sagemaker-core/venv/lib/python3.9/site-packages/botocore/credentials.py#1278\1278]8;;\
                             file: ~/.aws/credentials                                                              

ClientError: An error occurred (ValidationException) when calling the CreateCluster operation: Unable to retrieve security groups. Please ensure that the execution role allows the action sts:AssumeRole for the service principal sagemaker.amazonaws.com, and has all the permissions required to create a cluster.

In [ ]:
cluster.wait_for_status("InService")

### Using Resource Defaults
In the below example, a `TrainingJob` resource will be created using the `role` and `output_data_config` defined under the `SageMaker.Python.Resources.TrainingJob` key. 

Note: Because `TrainingJob` also excepts a `vpc_config` parameter, the `vpc_config` parameter will be populated from the `GlobalDefaults`

In [ ]:
import time
from sagemaker_core.generated.resources import TrainingJob
from sagemaker_core.generated.shapes import  AlgorithmSpecification, Channel, DataSource, S3DataSource, ResourceConfig, StoppingCondition

job_name_v3 = 'xgboost-iris-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

# Use role and output_data_config from Intelligent Defaults JSON config file under the SageMaker.PythonSDK.Resources.TrainingJob key
# Use vpc_config from Intelligent Defaults JSON config file under the SageMaker.PythonSDK.Resources.GlobalDefaults key

training_job = TrainingJob.create(
    training_job_name=job_name_v3,
    hyper_parameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    algorithm_specification=AlgorithmSpecification(
        training_image=image,
        training_input_mode='File'
    ),
    input_data_config=[
        Channel(
            channel_name='train',
            content_type='csv',
            compression_type='None',
            record_wrapper_type='None',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri=s3_input_path,
                    s3_data_distribution_type='FullyReplicated'
                )
            )
        )
    ],
    resource_config=ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_gb=30
    ),
    stopping_condition=StoppingCondition(
        max_runtime_in_seconds=600
    )
)

## Configure Logging Levels

Below are 2 examples of how a SageMakerCore user could configure the logging level of the SDK to assist with debugging.

To set the logging level users have 2 options:
1. Pass a string parameter to utility method with log level they desire - `configure_logging("DEBUG")`
2. Set the `LOG_LEVEL=INFO` environment variable and call `configure_logging()` without a parameter

### Configure Logging with Parameter

In [ ]:
# Setting log_level to DEBUG using configure_logging with string parameter 
from sagemaker_core.generated.utils import configure_logging

configure_logging('DEBUG')

In [ ]:
# Get TrainingJob with DEBUG log_level
from sagemaker_core.generated.resources import TrainingJob

training_job = TrainingJob.get(job_name_v3)

### Configure Logging with Enviornment Variable

In [ ]:
# Setting log_level to INFO using env variable
!export LOG_LEVEL=INFO

configure_logging()

In [ ]:
# List TrainingJobs with INFO log_level
training_job = TrainingJob.get(job_name_v3)

## Delete All SageMaker Resources
The following code block will call the delete() method for any SageMaker Core Resources created during the execution of this notebook which were assigned to local or global variables. If you created any additional deleteable resources without assigning the returning object to a unique variable, you will need to delete the resource manually by doing something like:

```python
resource = Resource.get("resource-name")
resource.delete()
```


In [ ]:
# Delete any sagemaker core resource objects created in this notebook
def delete_all_sagemaker_resources():
    all_objects = list(locals().values()) + list(globals().values())
    deletable_objects = [obj for obj in all_objects if hasattr(obj, 'delete') and obj.__class__.__module__ == 'sagemaker_core.generated.resources']
    
    for obj in deletable_objects:
        obj.delete()
        
delete_all_sagemaker_resources()